In [ ]:
import torch
torch.cuda.is_available()

## Depth -> pos using camera 

In [ ]:
from core.options import AllConfigs 
from core.provider_objaverse import ObjaverseDataset as Dataset
import tyro
import time
import random

import torch
from core.options import AllConfigs
# from core.models import LGM
from core.models_fix_pretrained import LGM
from accelerate import Accelerator, DistributedDataParallelKwargs
from safetensors.torch import load_file

import kiui

from ipdb import set_trace as st
import os
import json
import re
import glob 
import shutil
from tqdm import tqdm
import numpy as np
import sys



In [ ]:


data_path="/mnt/kostas-graid/sw/envs/chenwang/workspace/lrm-zero123/assets/9000-9999"
scene_name_pattern = os.path.join(data_path, '*')
scene_dirs = sorted(glob.glob(scene_name_pattern))
for i, scene_path in enumerate(scene_dirs):
    if i > 3:
        break
    scene_name = scene_path.split('/')[-2] if scene_path.endswith('/') else scene_path.split('/')[-1]
    print(scene_name)


In [ ]:


 # Filter out the --f argument if it's passed
filtered_args = ["big"]
# filtered_args += [arg for arg in sys.argv if not arg.startswith('--f=')]
# print(filtered_args)

opt = tyro.cli(AllConfigs, args=filtered_args)
# print(opt)

 # training 
train_dataset = Dataset(opt, name=scene_path, training=True)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=opt.batch_size,
    shuffle=False,
    # num_workers=opt.num_workers,
    num_workers=0,
    pin_memory=True,
    drop_last=False,
)

In [ ]:

for i, data in enumerate(train_dataloader):
    if i > 1 :
        break
    print(data.keys())
    print(data['cam_pos'].shape)
    print(data['cam_view'].shape)

    # use cam pos for projection
    print(data['cam_view_proj'])


In [ ]:
cam_index = 2
cam_view = data['cam_view'][:,cam_index] # choose one cam
cam_poses = data['c2w'][:,cam_index] # choose one cam
cam_view.shape, cam_poses.shape
# cam_poses[:, :3, 1:3] *= -1
cam_poses_neg = cam_poses.clone()
cam_poses_neg[:, :3, 1:3] *= -1

cam_view2 = torch.inverse(cam_poses).transpose(1, 2)
cam_view3 = torch.inverse(cam_poses_neg).transpose(1, 2)
torch.allclose(cam_view, cam_view2), torch.allclose(cam_view, cam_view3)


In [ ]:
# check whether bmm is the same as @ 
N=20
# c2w = cam_poses
c2w = cam_poses_opengl
pos = torch.rand(1,N,3)
cam_pos = torch.cat([pos,
           torch.ones((pos.shape[0], pos.shape[1], 1), device=c2w.device, dtype=torch.float32)
            ], dim=2)
pos_bmm = torch.bmm(cam_pos, c2w)
pos_bmm_transpose = torch.bmm(cam_pos, c2w.transpose(1,2))
pos_at = pos @ c2w[:, :3, :3].transpose(1,2) + c2w[:, :3, 3]
pos_left = (c2w @ cam_pos.transpose(1,2)).transpose(1,2) # pos_at = pos_left = pos_bmm_transpose
# 

In [ ]:
pos_left = pos_left[...,:3] / pos_left[...,3:]
pos_left

In [ ]:

# pos_bmm = pos_bmm[...,:3] / pos_bmm[...,3:]
# pos_bmm_transpose = pos_bmm_transpose[...,:3] / pos_bmm_transpose[...,3:]
pos_bmm, 
pos_bmm_transpose

In [ ]:
pos_at

In [ ]:
torch.allclose(pos_at, pos_left), torch.allclose(pos_at, pos_bmm_transpose), torch.allclose(pos_bmm, pos_left)

### check the einops consistency

In [ ]:

import einops

In [ ]:
B,V,H,W = 1, 6, 128, 128
depth = torch.rand([B,V,1,H,W])
depth_copy = depth.clone()
print(depth.shape)
depth = einops.rearrange(depth, 'b v c h w-> (b v) (h w) c', v=V, h=H, w=W)
depth = einops.rearrange(depth, '(b v) (h w) c-> b v c h w', v=V, h=H, w=W)

torch.allclose(depth_copy, depth)

### Verify the depth_act used in the splatter image

In [ ]:

import torch

In [ ]:
depth = torch.rand(6,213,2)*10
depth, torch.sigmoid(depth), torch.logit(torch.sigmoid(depth)), \
torch.allclose(depth,  torch.logit(torch.sigmoid(depth)))

### Save a vis of depth

In [ ]:
import torch
import kiui

In [ ]:
# read ply
# Load the tensor from the file
# file_path = 'runs/LGM_optimize_splatter/debug2/00013-depth_offset-es10-Plat-patience_20-factor_0.5-eval_5-adamW-subset_0_-1_splat128-inV6-lossV20-lr0.006/9000-9999/0047fa5e44334d3e9bd171ad2bf95ff5/eval_pred_gs_600_0/raw_splatter_out_tensor.pt'
# file_path = 'runs/LGM_optimize_splatter/debug2/00013-depth_offset-es10-Plat-patience_20-factor_0.5-eval_5-adamW-subset_0_-1_splat128-inV6-lossV20-lr0.006/9000-9999/0047fa5e44334d3e9bd171ad2bf95ff5/eval_pred_gs_1100_0_es/raw_splatter_out_tensor.pt'
# file_path = 'runs/LGM_optimize_splatter/debug2/00013-depth_offset-es10-Plat-patience_20-factor_0.5-eval_5-adamW-subset_0_-1_splat128-inV6-lossV20-lr0.006/9000-9999/0047fa5e44334d3e9bd171ad2bf95ff5/eval_pred_gs_0_0/raw_splatter_out_tensor.pt'
# file_path = 'runs/LGM_optimize_splatter/debug2/00013-depth_offset-es10-Plat-patience_20-factor_0.5-eval_5-adamW-subset_0_-1_splat128-inV6-lossV20-lr0.006/9000-9999/01529e7c8d7842c498f28a9d5927a0d6/eval_pred_gs_1410_0_es/raw_splatter_out_tensor.pt'
file_path = 'runs/LGM_optimize_splatter/debug2/00013-depth_offset-es10-Plat-patience_20-factor_0.5-eval_5-adamW-subset_0_-1_splat128-inV6-lossV20-lr0.006/9000-9999/00e90648c8b94d0b9ea8a92858bd8e73/eval_pred_gs_1999_0/raw_splatter_out_tensor.pt'
loaded_tensor = torch.load(file_path)

print(loaded_tensor.shape)
# get_depth and xyz offset from world pos

In [ ]:

# B, V, C, H, W = loaded_tensor.shape
# for b in range(B):
#     for v in range(V):       
#         depth = loaded_tensor[b, v, 14:,... ]
#         depth = depth.permute(1,2,0)
#         print(depth.shape)
#         kiui.write

depth = loaded_tensor[:,:,14:] 
print(depth.min(), depth.max(), depth.mean(), depth.median())
d_min, d_max = depth.min(), depth.max()
# d_min, d_max = 0.5, 2
depth = (depth - d_min) / (d_max - d_min)
gt_images = depth.detach().cpu().numpy() # [B, V, 3, output_size, output_size]
gt_images = gt_images.transpose(0, 3, 1, 4, 2).reshape(-1, gt_images.shape[1] * gt_images.shape[3], 1) # [B*output_size, V*output_size, 3]
kiui.write_image('depth_epoch0.jpg', gt_images)
        

In [ ]:
opacity = loaded_tensor[:,:, 3:4] 
print(opacity.min(), opacity.max(), opacity.mean(), opacity.median())
# d_min, d_max = opacity.min(), opacity.max()

d_min, d_max = -15, 5
depth = (opacity - d_min) / (d_max - d_min)
depth = 1 - depth.clamp(0,1)
gt_images = depth.detach().cpu().numpy() # [B, V, 3, output_size, output_size]
gt_images = gt_images.transpose(0, 3, 1, 4, 2).reshape(-1, gt_images.shape[1] * gt_images.shape[3], 1) # [B*output_size, V*output_size, 3]
kiui.write_image('opacity_epoch_bucket.jpg', gt_images)

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.cm as cm

depth_colormap = cm.jet(gt_images)  # This will produce an RGBA image
print(gt_images.shape, depth_colormap.shape)
# Remove the alpha channel
depth_colormap = depth_colormap[..., 0, :3]
plt.imsave('depth_colormap_birdman.png', depth_colormap)

# If you need to display the colorized depth map instead of saving it
plt.imshow(depth_colormap)
plt.axis('off')  # Disable axis
plt.show()

## Marigold 

In [ ]:
import torch
torch.cuda.is_availabl()

In [ ]:

import tyro
import time
import random

import torch
from core.options import AllConfigs
from core.models_zero123plus import Zero123PlusGaussian, gt_attr_keys, start_indices, end_indices, fuse_splatters
from core.models_zero123plus_code import Zero123PlusGaussianCode
from core.models_zero123plus_code_unet_lora import Zero123PlusGaussianCodeUnetLora

from core.models_fix_pretrained import LGM

from accelerate import Accelerator, DistributedDataParallelKwargs
from safetensors.torch import load_file
from core.dataset_v4_batch import ObjaverseDataset as Dataset

import kiui
from datetime import datetime
import torch.utils.tensorboard as tensorboard
import shutil, os

from ipdb import set_trace as st
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm
import torch.nn.functional as F

import warnings
from accelerate.utils import broadcast
import re

import numpy
from PIL import Image
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
# from zero123plus.img_to_mv_v3_my_decoder import to_rgb_image, unscale_image, unscale_latents

import einops
import rembg
import requests

from pytorch3d.transforms import quaternion_to_axis_angle, axis_angle_to_quaternion

In [ ]:

def to_rgb_image(maybe_rgba: Image.Image):
    if maybe_rgba.mode == 'RGB':
        return maybe_rgba
    elif maybe_rgba.mode == 'RGBA':
        rgba = maybe_rgba
        img = numpy.random.randint(127, 128, size=[rgba.size[1], rgba.size[0], 3], dtype=numpy.uint8)
        img = Image.fromarray(img, 'RGB')
        img.paste(rgba, mask=rgba.getchannel('A'))
        return img
    else:
        raise ValueError("Unsupported image type.", maybe_rgba.mode)

def unscale_latents(latents):
    latents = latents / 0.75 + 0.22
    return latents

def unscale_image(image):
    image = image / 0.5 * 0.8
    return image

def scale_image(image):
    image = image * 0.5 / 0.8
    return image

def scale_latents(latents):
    latents = (latents - 0.22) * 0.75
    return latents

def normalize_to_target(source_tensor, target_tensor):
    # Calculate mean and standard deviation of source tensor
    source_mean = torch.mean(source_tensor)
    source_std = torch.std(source_tensor)

    # Calculate mean and standard deviation of target tensor
    target_mean = torch.mean(target_tensor)
    target_std = torch.std(target_tensor)

    # Normalize source tensor to target distribution
    normalized_tensor = (source_tensor - source_mean) / source_std * target_std + target_mean

    return normalized_tensor

def fuse_splatters(splatters):
    # fuse splatters
    B, V, C, H, W = splatters.shape

    x = splatters.permute(0, 1, 3, 4, 2).reshape(B, -1, 14)
    return x


### Config

In [ ]:
filtered_args = ["big"]
opt = tyro.cli(AllConfigs,  args=filtered_args)

from core.options import config_defaults
big_config = config_defaults['big']

# Now, big_config is an instance of Options with the 'big' configuration preset.
# You can use it directly or modify it as needed
big_config.num_epochs = 50  # Modify the number of epochs if needed
big_config.batch_size = 1  # Change the batch size if needed

print(opt)
# Merge the 'big' config with the parsed command-line arguments
for key, value in vars(opt).items():
    if hasattr(big_config, key) and value is not None:
        setattr(big_config, key, value)


## modify the command line args
DATA_DIR_BATCH_RENDERING='/mnt/kostas-graid/sw/envs/chenwang/workspace/lrm-zero123/assets/9000-9999'
DATA_DIR_BATCH_SPLATTER_GT_ROOT='/home/xuyimeng/Repo/LGM/runs/splatter_gt'

# Now modify opt based on the command line arguments
opt.workspace = "runs/zerp123plus_batch/workspace_inference2"
opt.lr = 2e-4
opt.num_epochs = 10001
opt.eval_iter = 5
opt.save_iter = 5
opt.lr_scheduler = "Plat"
opt.lr_scheduler_patience = 100
opt.lr_scheduler_factor = 0.7
opt.prob_cam_jitter = 0
opt.input_size = 320
opt.num_input_views = 6
opt.num_views = 20
opt.lambda_splatter = 0
opt.lambda_rendering = 1
opt.lambda_alpha = 0
opt.lambda_lpips = 0
opt.desc = "debug_encode_splatter"
opt.data_path_rendering = DATA_DIR_BATCH_RENDERING  # Replace with actual variable or constant
opt.data_path_splatter_gt = DATA_DIR_BATCH_SPLATTER_GT_ROOT  # Replace with actual variable or constant
opt.set_random_seed = True  # This implies there's some handling in the code to manage random seeds
opt.batch_size = 1
opt.num_workers = 1
opt.plot_attribute_histgram = ('scale',)  # Assuming this accepts a tuple of strings
opt.skip_predict_x0 = True
opt.scale_act = "biased_softplus"
opt.scale_act_bias = -3.
opt.scale_bias_learnable = True
opt.scale_clamp_max = -2
opt.scale_clamp_min = -10
opt.model_type = "Zero123PlusGaussianCode"
opt.splatter_guidance_interval = 1
opt.save_train_pred = -1
opt.decode_splatter_to_128 = True
opt.decoder_upblocks_interpolate_mode = "last_layer"
opt.codes_from_diffusion = True
opt.vae_on_splatter_image = True
opt.overfit_one_scene = True

print(opt)


### Load diffusion pipeline

In [ ]:
# only use model for model.gs.render()

model = Zero123PlusGaussianCode(opt)
from core.dataset_v4_code import ObjaverseDataset as Dataset


In [ ]:

# fix this VAE encoder and decoder, find a good latent  

from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
# pipe = DiffusionPipeline.from_pretrained(
#         "sudo-ai/zero123plus-v1.1", custom_pipeline="/mnt/kostas-graid/sw/envs/chenwang/workspace/diffgan/training/modules/zero123plus.py",
#         torch_dtype=torch.float32
#     )
pipe = DiffusionPipeline.from_pretrained(
    "sudo-ai/zero123plus-v1.1", custom_pipeline="sudo-ai/zero123plus-pipeline",
    torch_dtype=torch.float32
)
pipe.to('cuda:0')
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(
    pipe.scheduler.config, timestep_spacing='trailing'
)
output_path = f"{opt.workspace}/zero123plus/outputs_v3_inference_my_decoder"
pipe.prepare()
guidance_scale = 4.0

### debug rembg

In [ ]:
path = f"{DATA_DIR_BATCH_RENDERING}/ffb0d644238b4c679658aa0ee46ac6da"
path

In [ ]:

poses = [np.load(f'{path}/{i:03d}.npy', allow_pickle=True).item() for i in range(1, 56)]
elevations, azimuths = [-pose['elevation'] for pose in poses], [pose['azimuth'] for pose in poses]
# imgs = [np.array(Image.open(f'{path}/{i:03d}.png')) / 255.0 for i in range(1, 7)]
# imgs = [img[..., :3] * img[..., 3:4] + (1 - img[..., 3:4]) for img in imgs]

# get imgs from pipeline
cond = Image.open(f'{path}/{0:03d}.png')
from IPython.display import display
display(cond, )

In [ ]:

result = pipe(cond, num_inference_steps=75).images[0]

from torchvision import transforms
IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)

# Normalization typically required for pre-trained models
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Combine ToTensor and Normalize transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    # normalize
])

image = transform(result) # 3, 960, 640
mv_image = einops.rearrange((image.clip(0,1)).cpu().numpy()*255, 'c (h2 h) (w2 w)-> (h2 w2) h w c', h2=3, w2=2).astype(np.uint8)

In [ ]:
def prepare_default_rays( device, elevation, azimuth):
        
    from kiui.cam import orbit_camera
    from core.utils import get_rays

    # cam_poses = np.stack([
    #     orbit_camera(-30, 30, radius=self.opt.cam_radius),
    #     orbit_camera(20, 90, radius=self.opt.cam_radius),
    #     orbit_camera(-30, 150, radius=self.opt.cam_radius),
    #     orbit_camera(20, 210, radius=self.opt.cam_radius),
    #     orbit_camera(-30, 270, radius=self.opt.cam_radius),
    #     orbit_camera(20, 330, radius=self.opt.cam_radius),
    # ], axis=0) # [4, 4, 4]
    cams = [orbit_camera(ele, azi, radius=opt.cam_radius) for (ele, azi) in zip(elevation, azimuth)]
    cam_poses = np.stack(cams, axis=0)
    cam_poses = torch.from_numpy(cam_poses)

    rays_embeddings = []
    for i in range(cam_poses.shape[0]):
        rays_o, rays_d = get_rays(cam_poses[i], opt.input_size, opt.input_size, opt.fovy) # [h, w, 3]
        rays_plucker = torch.cat([torch.cross(rays_o, rays_d, dim=-1), rays_d], dim=-1) # [h, w, 6]
        rays_embeddings.append(rays_plucker)

        ## visualize rays for plotting figure
        # kiui.vis.plot_image(rays_d * 0.5 + 0.5, save=True)

    rays_embeddings = torch.stack(rays_embeddings, dim=0).permute(0, 3, 1, 2).contiguous().to(device) # [V, 6, h, w]
    
    return rays_embeddings


input_image = torch.from_numpy(mv_image).permute(0, 3, 1, 2).float().to(pipe.device) # [4, 3, 256, 256]
input_image = F.interpolate(input_image, size=(opt.input_size, opt.input_size), mode='bilinear', align_corners=False)
# if opt.model_type == 'LGM':
#     rays_embeddings = model.prepare_default_rays(device, elevations[:6], azimuths[:6])
#     input_image = TF.normalize(input_image, IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)
#     input_image = torch.cat([input_image, rays_embeddings], dim=1).unsqueeze(0) # [1, 4, 9, H, W]
rays_embeddings = prepare_default_rays(pipe.device, elevations[:6], azimuths[:6])
input_image = torch.cat([input_image, rays_embeddings], dim=1).unsqueeze(0) # [1, 4, 9, H, W]


In [ ]:
from scipy.stats import mode
mode_result = mode(mv_image[0].reshape(-1,3))


In [ ]:
mode_result.mode + 1

# # remok.sve bg
# bg_mask_1 = np.all(mode_result.mode == mv_image[0], axis=-1) 
# bg_mask_2 = np.all((mode_result.mode + 1) == mv_image[0], axis=-1)
# # bg_mask = np.any(bg_mask_1 or bg_mask_2)
# bg_mask = (bg_mask_1 + bg_mask_2).clip(0,1)
# # bg_mask.shape, bg_mask.astype(np.uint8)

half_img = (mv_image[0].astype(np.float32) / 2. ).round().astype(np.uint8)
bg_mask = np.all(64 == half_img, axis=-1) 
# bg_mask_2 = np.all((mode_result.mode + 1) == mv_image[0], axis=-1)
plt.imshow(bg_mask)


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(mv_image[0])
plt.imshow(bg_mask.astype(np.uint8)*255)

img_no_bg = mv_image[0] * (1 - bg_mask[...,None]) + bg_mask[...,None]
plt.imshow(img_no_bg)

In [ ]:
mv_image_no_bg = rembg.remove(einops.rearrange(mv_image, 'b h w c -> h (b w) c')) # .astype(np.float32) / 255.

In [ ]:
# plt.imshow(img_no_bg)
# np.array([3.6]).round().astype(np.uint8)
plt.imshow(mv_image_no_bg[...,:-1])
# plt.imshow(einops.rearrange(mv_image, 'b h w c -> h (b w) c'))

In [ ]:
mv_image_no_bg.shape
mask = mv_image_no_bg[..., -1] > 0

# from kiui.op import recenter
# # recenter
# image = recenter(mv_image_no_bg, mask, border_ratio=0.2)

# generate mv
# image = image.astype(np.float32) / 255.0
image = mv_image_no_bg.astype(np.float32) / 255.0

### Load dataset

In [ ]:
train_dataset = Dataset(opt, training=True)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.num_workers,
    pin_memory=True,
    drop_last=False,
)

In [ ]:
for i, data in enumerate(train_dataloader):
   print(i)

In [ ]:
gt_images = einops.rearrange(data["input"], "b (m n) c h w -> (b m h) (n w) c", m=3, n=2).detach().cpu().numpy() # [B, V, 3, output_size, output_size]
# gt_images = gt_images.transpose(0, 3, 1, 4, 2).reshape(-1, gt_images.shape[1] * gt_images.shape[3], gt_images.shape[2]) # [B*output_size, V*output_size, 3]
kiui.write_image('data_input.jpg', gt_images)

data["input"].shape, gt_images.shape


In [ ]:
depth_file = "/home/xuyimeng/Repo/LGM/0_red_white_blue_gt_depth_fp32.npy"
depth_array_fp32 = np.load(depth_file)
depth_array_fp32.shape, depth_array_fp32.min(), depth_array_fp32.max() # (960, 640), 0.0, 1.0

In [ ]:
splatters_mv = einops.rearrange(data["splatters_output"], 'b (h2 w2) c h w -> b c (h2 h) (w2 w)', h2=3, w2=2) 

### 1. Bad init from LGM output

In [ ]:
 # process the loaded splatters into 3-channel images
gt_attr_keys = ['pos', 'opacity', 'scale', 'rotation', 'rgbs']
start_indices = [0, 3, 4, 7, 11]
end_indices = [3, 4, 7, 11, 14]
attr_map = {key: (si, ei) for key, si, ei in zip (gt_attr_keys, start_indices, end_indices)}
sp_min_max_dict = {"z-depth": (-0.7, 0.7), 
                   "xy-offset": (-0.7, 0.7), 
                   "pos": (-0.7, 0.7), 
                   "scale": (-10., -2.),
                   }

def prepare_3channel_images_to_encode(splatters_mv):
   
    splatter_3Channel_image = {}

    for attr_to_encode, (start_i, end_i) in attr_map.items():
    
        splatter_attr = splatters_mv[:, start_i:end_i,...]

        sp_min, sp_max = None, None

        # process the channels
        if end_i - start_i == 1:
            print(f"repeat attr {attr_to_encode} for 3 times")
            splatter_attr = splatter_attr.repeat(1, 3, 1, 1) # [0,1]
        elif end_i - start_i == 3:
            pass
        elif attr_to_encode == "xy-offset":
            # ## normalize to [0,1]
            # sp_min, sp_max =  -1., 1.
            # splatter_attr = (splatter_attr - sp_min) / (sp_max - sp_min)
            ## cat one more dim
            splatter_attr = torch.cat((splatter_attr, 0.5 * torch.ones_like(splatter_attr[:,0:1,...])), dim=1)
        elif attr_to_encode == "rotation":
            # st() # assert 4 is on the last dim
            # quaternion to axis angle
            quat = einops.rearrange(splatter_attr, 'b c h w -> b h w c')
            axis_angle = quaternion_to_axis_angle(quat)
            splatter_attr = einops.rearrange(axis_angle, 'b h w c -> b c h w')
            # st()

        else:
            raise ValueError(f"The dimension of {attr_to_encode} is problematic to encode")

        if "scale" in attr_to_encode:
            # use log scale
            splatter_attr = torch.log(splatter_attr)
            
            print(f"{attr_to_encode} log min={splatter_attr.min()} max={splatter_attr.max()}")
            sp_min, sp_max =  -10., -2.
            splatter_attr = (splatter_attr - sp_min) / (sp_max - sp_min) # [0,1]
            splatter_attr = splatter_attr.clip(0,1)

        elif attr_to_encode in ["z-depth", "xy-offset", "pos"] :
            # sp_min, sp_max =  splatter_attr.min(), splatter_attr.max()
            # sp_min, sp_max =  -1., 1.
            sp_min, sp_max =  -0.7, 0.7
            splatter_attr = (splatter_attr - sp_min) / (sp_max - sp_min)


        print(f"[{attr_to_encode}] in [0,1]: min={splatter_attr.min()} max={splatter_attr.max()}")

        sp_image = splatter_attr * 2 - 1 # [map to range [-1,1]]
        print(f"[{attr_to_encode}] in [-1, 1]: min={sp_image.min()} max={sp_image.max()}")
        
        splatter_3Channel_image[attr_to_encode] = sp_image

    return splatter_3Channel_image


In [ ]:

splatter_3Channel_image_returned = prepare_3channel_images_to_encode(splatters_mv)
print()
for attr_to_encode, splatter_attr in splatter_3Channel_image_returned.items():
    print(attr_to_encode, splatter_attr.shape, splatter_attr.min(), splatter_attr.max())


                        

In [ ]:
name = "pipe_vae"

In [ ]:
# feed the 3-channel splatter images into encoder for init latent code
# Save image before encoding
def encode_3channel_image_to_latents(sp_image):
    
    if not os.path.exists(os.path.join(output_path, name)):
        os.makedirs(os.path.join(output_path, name), exist_ok=True)
    mv_image = einops.rearrange((sp_image[0].clip(-1,1)+1).cpu().numpy()*127.5, 'c h w-> h w c').astype(np.uint8) 
    Image.fromarray(mv_image).save(os.path.join(output_path, f'{name}/{attr_to_encode}_to_encode.png'))

    # encode: splatter attr -> latent 
    # sp_image_original = sp_image.clone()
    sp_image = scale_image(sp_image.to(pipe.device))
    sp_image = pipe.vae.encode(sp_image).latent_dist.sample() * pipe.vae.config.scaling_factor
    latents = scale_latents(sp_image)

    return latents


In [ ]:
## get latents by encoding 

latents_all_attr_dict = {}
for attr in gt_attr_keys:
    latents_single_attr = encode_3channel_image_to_latents(splatter_3Channel_image_returned[attr])
    latents_all_attr_dict.update({attr: latents_single_attr.detach().clone().requires_grad_(True)})

latents_all_attr_tensor = torch.cat([latents_all_attr_dict[attr] for attr in gt_attr_keys])
print(latents_all_attr_tensor.shape, len(gt_attr_keys)) # torch.Size([5, 4, 48, 32])
assert latents_all_attr_tensor.shape[0] == len(gt_attr_keys)

In [ ]:

# save the latent code for optimization

import torch
from torch.optim import Adam

latent_codes = latents_all_attr_tensor
# optimizer = Adam([latent_codes], lr=0.01)
optimizer = Adam([latents_all_attr_dict[attr] for attr in gt_attr_keys], lr=0.01)


# Optimization loop
num_iterations = 10
acceptable_loss_threshold = 1

decoded_attr_image_dict = {}

for i in range(num_iterations):
    print(f"[Iter {i}]")
    # Generate images from updated latent codes
    # Step 2: Render the images from the latents
    # Reshape latents if necessary and decode them
    for attr, latents in latents_all_attr_dict.items():
        if attr not in ["scale"]:
            decoded_attr_image_dict.update({attr:decoded_images})
            start_i, end_i = attr_map[attr]
            splatter_attr = splatters_mv[:, start_i:end_i,...]
            decoded_attr_image_dict.update({attr:splatter_attr.to(pipe.device)})
            continue
        if latents.requires_grad and latents.grad is not None:
            print(f"Gradient of {attr}: {latents.requires_grad} -- {latents.grad.norm().item()}")

        decoded_images = decode_single_latents(latents, attr_to_encode=attr)
        decoded_attr_image_dict.update({attr:decoded_images})
    
   
    # order the attributes back into splatter image
    splatters_to_render = get_splatter_images_from_decoded_dict(decoded_attr_image_dict, group_scale=True)
    
    
    
    # Render the decoded images using the splatter model
    gs_results = render_from_decoded_images(splatters_to_render, data=data)
    
    
    # Step 3: Calculate the loss
    # The target images should be the ground truth images you are trying to approximate
    loss = calculate_loss(gs_results, data)
    break
    
    # Step 4: Backward pass
    loss.backward()  # Compute gradient of the loss w.r.t. latents
    
    # Step 5: Update the latents
    optimizer.step()  # Perform a single optimization step
    # check whether each latents has gradients
 
    # Step 6: Print the loss or log it to observe convergence
    print(f'Iteration {i}, Loss: {loss.item()}')

    # Check for convergence or a stopping condition
    if loss.item() < acceptable_loss_threshold:
        print("Lower than acceptable_loss_threshold, success!")
        break


In [ ]:
def decode_single_latents(latents, attr_to_encode):
    start_i, end_i = attr_map[attr_to_encode]

    #  decode: latents -> platter attr
    latents1 = unscale_latents(latents)
    image = pipe.vae.decode(latents1 / pipe.vae.config.scaling_factor, return_dict=False)[0]
    image = unscale_image(image)


    # save decoded image
    mv_image_numpy = einops.rearrange((image[0].clip(-1,1)+1).detach().cpu().numpy()*127.5, 'c h w-> h w c').astype(np.uint8) 
    Image.fromarray(mv_image_numpy).save(os.path.join(output_path, f'{name}/{attr_to_encode}_pred.png'))

    # scale back to original range
    mv_image = image # [b c h w], in [-1,1]

    # if attr_to_encode in[ "z-depth", "xy-offset"]:
    #     sp_image_o = mv_image
    #     # st() # NOTE: try clip z-depth? No need, they are already within the range
    # else:
    sp_image_o = 0.5 * (mv_image + 1) # [map to range [0,1]]

    print(f"Decoded attr [-1,1] {attr_to_encode}: min={mv_image.min()} max={mv_image.max()}")
    print(f"Decoded attr [0,1] {attr_to_encode}: min={sp_image_o.min()} max={sp_image_o.max()}")

    if "scale" in attr_to_encode:
        # v2
        sp_min, sp_max = sp_min_max_dict["scale"]

        sp_image_o = sp_image_o.clip(0,1) 
        sp_image_o = sp_image_o * (sp_max - sp_min) + sp_min
        
        print(f"Decoded attr not clip [0,1] {attr_to_encode}: min={sp_image_o.min()} max={sp_image_o.max()}")
        sp_image_o = torch.exp(sp_image_o)
        print(sp_min, sp_max)
        print(f"Decoded attr [unscaled] {attr_to_encode}: min={sp_image_o.min()} max={sp_image_o.max()}")

    # elif attr_to_encode == "z-depth":
        # sp_image_o = sp_image_o * (sp_max - sp_min) + sp_min

    elif attr_to_encode in[ "z-depth", "xy-offset", "pos"]:
        sp_min, sp_max = sp_min_max_dict[attr_to_encode]
        sp_image_o = sp_image_o * (sp_max - sp_min) + sp_min

    if attr_to_encode == "xy-offset": 
        sp_image_o = sp_image_o[:,:2] # FIXME: ...,2??

    if attr_to_encode == "rotation": 
        
        ag = einops.rearrange(sp_image_o, 'b c h w -> b h w c')
        quaternion = axis_angle_to_quaternion(ag)
        sp_image_o = einops.rearrange(quaternion, 'b h w c -> b c h w')
        # st()

    if end_i - start_i == 1:
        # print(torch.allclose(torch.mean(sp_image_o, dim=1, keepdim=True), sp_image_o))
        # st()
        print(f"Decoded attr [unscaled, before mean] {attr_to_encode}: min={sp_image_o.min()} max={sp_image_o.max()}")
        sp_image_o = torch.mean(sp_image_o, dim=1, keepdim=True) # avg.


        print(f"Decoded attr [unscaled, after median] {attr_to_encode}: min={sp_image_o.min()} max={sp_image_o.max()}")
        
  
    print(f"Decoded attr [unscaled] {attr_to_encode}: min={sp_image_o.min()} max={sp_image_o.max()}")
    
    return sp_image_o


In [ ]:
def get_splatter_images_from_decoded_dict(decoded_attr_image_dict, group_scale=False):
        
    if not group_scale:
        ordered_attr_list = ["xy-offset", "z-depth", # 0-3
                            'opacity', # 3-4
                            'scale-x', 'scale-y', 'scale-z', # 4-7
                            "rotation", # 7-11
                            "rgbs", # 11-14
                            ] # must be an ordered list according to the channels
    else:
        ordered_attr_list = ["pos", # 0-3
                            'opacity', # 3-4
                            'scale', # 4-7
                            "rotation", # 7-11
                            "rgbs", # 11-14
                            ] # must be an ordered list according to the channels
    
    attr_image_list = [decoded_attr_image_dict[attr] for attr in ordered_attr_list ]
    # [print(t.shape) for t in attr_image_list]
    splatter_mv = torch.cat(attr_image_list, dim=1)

    ## reshape 
    splatters_to_render = einops.rearrange(splatter_mv, 'b c (h2 h) (w2 w) -> b (h2 w2) c h w', h2=3, w2=2) 

    return splatters_to_render
                    

In [ ]:
def render_from_decoded_images(splatters_to_render, data):
    gaussians = fuse_splatters(splatters_to_render)
    bg_color = torch.ones(3, dtype=torch.float32, device=gaussians.device) * 0.5
    gs_results = model.gs.render(gaussians, data['cam_view'], data['cam_view_proj'], data['cam_pos'], bg_color=bg_color)
                    
    return gs_results

def calculate_loss(gs_results, data):
    results = {}
    loss = 0
    
    pred_images = gs_results['image'] # [B, V, C, output_size, output_size]
    pred_alphas = gs_results['alpha'] # [B, V, 1, output_size, output_size]

    gt_images = data['images_output'] # [B, V, 3, output_size, output_size], ground-truth novel views
    gt_masks = data['masks_output'] 

    gt_images = gt_images * gt_masks + (1 - gt_masks) # NOTE: white bg

    loss_mse_rendering = F.mse_loss(pred_images, gt_images) + F.mse_loss(pred_alphas, gt_masks)
    results['loss_rendering'] = loss_mse_rendering

    loss_lpips = model.lpips_loss(
    # gt_images.view(-1, 3, self.opt.output_size, self.opt.output_size) * 2 - 1,
    # pred_images.view(-1, 3, self.opt.output_size, self.opt.output_size) * 2 - 1,
    # downsampled to at most 256 to reduce memory cost
    
    # FIXME: change the dim to 14 for splatter imaegs
    F.interpolate(gt_images.view(-1, 3, opt.output_size, opt.output_size) * 2 - 1, (256, 256), mode='bilinear', align_corners=False), 
    F.interpolate(pred_images.view(-1, 3, opt.output_size, opt.output_size) * 2 - 1, (256, 256), mode='bilinear', align_corners=False),
    ).mean()
    results['loss_lpips'] = loss_lpips
    loss = loss + opt.lambda_lpips * loss_lpips
  
    print(f"loss lpips:{loss_lpips}")
        
    return loss

In [ ]:
import torch

# Clear CUDA memory
torch.cuda.empty_cache()


In [ ]:
splatter_attr.device
#  os.path.exists(os.path.join(output_path, name))

### 2. Fake init from RGB + depth + constant others

In [ ]:
## rgb
gt_images = einops.rearrange(data["input"], "b (m n) c h w -> (b m h) (n w) c", m=3, n=2).detach().cpu().numpy() # [B, V, 3, output_size, output_size]
# gt_images = gt_images.transpose(0, 3, 1, 4, 2).reshape(-1, gt_images.shape[1] * gt_images.shape[3], gt_images.shape[2]) # [B*output_size, V*output_size, 3]
kiui.write_image('data_input.jpg', gt_images)

data["input"].shape, gt_images.shape


In [ ]:
# depth 
## download marigold demo file
depth_file = "/home/xuyimeng/Repo/LGM/0_red_white_blue_gt_depth_fp32.npy"
depth_array_fp32 = np.load(depth_file)
depth_array_fp32.shape, depth_array_fp32.min(), depth_array_fp32.max() # (960, 640), 0.0, 1.0

In [ ]:
!pip install ipywidgets==7.7.1 --quiet

In [ ]:
# depth 
## load meigold pipeline
from IPython.display import display
import ipywidgets as widgets
import shutil

ckpt_dic = {
    "Original (higher quality)": "prs-eth/marigold-v1-0",
    "LCM (faster)": "prs-eth/marigold-lcm-v1-0",
}

ckpt_name = 'LCM (faster)'
ckpt_path = ckpt_dic[ckpt_name]
w = widgets.Dropdown(
    options=['Original (higher quality)', 'LCM (faster)'],
    value=ckpt_name,
    description='Checkpoint:',
)


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        ckpt_name = change['new']
        ckpt_path = ckpt_dic[ckpt_name]
        # print(f"Change to checkpoint: {ckpt_name} ({ckpt_path})")

w.observe(on_change)

display(w)

In [ ]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained(
    ckpt_path,
    custom_pipeline="marigold_depth_estimation"
)

pipe = pipe.to("cuda")